In [12]:
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

######################################################################
dataset = np.loadtxt('stpv_pv_20k.csv', delimiter=',')
feature_names = ['d1', 'd2', 'd3', 'd4', 'am_absorb', 'swir_emit']

X = dataset[:,0:4]
y1 = dataset[:,4]
y2 = dataset[:,5]

# split into train and test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y1, test_size=0.33)


############################################################
### Decision tree
tree = DecisionTreeRegressor()
tree.fit(X_train, y_train)



DecisionTreeRegressor()

In [21]:
import wpspecdev
# define a test structure
test_args = {
    "therml": True,
    'TeMperature' : 440,
    "wavelength_list": [300e-9, 20300e-9, 1001],
    "material_list": ['Air', 'SiO2', 'Al', 'SiO2', 'Ta2O5', 'polystyrene', 'SiO2', 'Al', 'SiO2', 'Ta2O5', 'Air'], 
    "thickness_list": [0, 2.64e-07, 1.5e-09, 2.64e-07, 3.73e-07,  0.001, 3.80e-07, 1.5e-09, 3.80e-07, 5.31e-07, 0]
}


sf = wpspecdev.SpectrumFactory()  

# create instance of class
ts = sf.spectrum_factory("Tmm", test_args)

# want to create a step function so that we can only integrate
# a subinterval of the entire reflectivity function up to 760 nm
target_pv = 760e-9 * np.ones_like(ts.wavelength_array)
stop_idx = np.argmin(np.abs( target_pv - ts.wavelength_array) )

# step function to cut off at \lambda_bg of the PV
step_pv = np.zeros_like(ts.wavelength_array)
step_pv[:stop_idx] = 1.

# do the same for the to create a step function between 1000 and 3500 nm
target_min = 1000e-9 * np.ones_like(ts.wavelength_array)
target_max = 3500e-9 * np.ones_like(ts.wavelength_array)

min_idx = np.argmin(np.abs( target_min - ts.wavelength_array))
max_idx = np.argmin(np.abs( target_max - ts.wavelength_array))

step_swir = np.zeros_like(ts.wavelength_array)
step_swir[min_idx:max_idx] = 1.


### Integrate the above-gap solar flux transmitted through optic
AM_transmit_power = np.trapz(ts._solar_spectrum * step_pv *  ts.transmissivity_array, ts.wavelength_array)
### Integrate the thermal emission of the optic in the SWIR
SWIR_emit_power = np.trapz(ts.thermal_emission_array * step_swir, ts.wavelength_array)

### Integrate the total above-gap solar flux
AM_above_gap = np.trapz(ts._solar_spectrum * step_pv, ts.wavelength_array)
### Integrate the total BB emission at 440 K in the SWIR
SWIR_power = np.trapz(ts.blackbody_spectrum * step_swir, ts.wavelength_array)


print(SWIR_emit_power,  " W / s / m^2 emitted")
print(AM_transmit_power," W / s / m^2 transmitted")

print(SWIR_power,  " W / s / m^2 emitted")
print(AM_above_gap," W / s / m^2 transmitted")


getting solar spectrum
 Your spectra have been computed! 😎 
 Your spectra have been computed! 🔥 
6.56263602124357  W / s / m^2 emitted
288.4765013494398  W / s / m^2 transmitted
10.200643508913217  W / s / m^2 emitted
461.7291630999993  W / s / m^2 transmitted


In [76]:
import random
def update_multilayer(SP, AP, d):
    # scale d so that each entry is in range [0, 700nm)
    ts.thickness_array[3] = d[0] * 700e-9
    ts.thickness_array[4] = d[1] * 700e-9
    ts.thickness_array[8] = d[2] * 700e-9
    ts.thickness_array[9] = d[3] * 700e-9
    
    # update spectra
    ts.compute_spectrum()
    ts._compute_therml_spectrum(ts.wavelength_array, ts.emissivity_array)
    AM_transmit_power = np.trapz(ts._solar_spectrum * step_pv *  ts.transmissivity_array, ts.wavelength_array)
    ### Integrate the thermal emission of the optic in the SWIR
    SWIR_emit_power = np.trapz(ts.thermal_emission_array * step_swir, ts.wavelength_array)
    return AM_transmit_power/AP, SWIR_emit_power/SP

In [77]:
def objective_exact(x0):
    print(x0)
    a, b = update_multilayer(SWIR_power, AM_above_gap, x0)
    return -a

def objective_ml(x0):
    print(x0)
    x_test = x0.reshape(1, -1)
    print(x_test)
    a = tree.predict(x_test)
    return -a[0]

    
    

In [83]:
d = np.random.rand(4)
print(d)


[0.9795551  0.9411697  0.13897678 0.1824032 ]


In [84]:
print(objective_exact(d))

[0.9795551  0.9411697  0.13897678 0.1824032 ]
-0.6037389389707041


In [85]:
print(objective_ml(d))

[0.9795551  0.9411697  0.13897678 0.1824032 ]
[[0.9795551  0.9411697  0.13897678 0.1824032 ]]
-0.6060185303044683


In [86]:
from scipy.optimize import minimize
 
bfgs_xmin = np.ones(4) * 0.001428
bfgs_xmax = np.ones(4)

# rewrite the bounds in the way required by L-BFGS-B
bfgs_bounds = [(low, high) for low, high in zip(bfgs_xmin, bfgs_xmax)]

# perform the search
result = minimize(objective_exact, d, method='L-BFGS-B', jac=False, bounds=bfgs_bounds)
# summarize the result
print('Status : %s' % result['message'])
print('Total Evaluations: %d' % result['nfev'])
# evaluate solution
solution = result['x']
evaluation = objective_exact(solution)
print('Solution: f(%s) = %.5f' % (solution, evaluation))

[0.9795551  0.9411697  0.13897678 0.1824032 ]
[0.97955511 0.9411697  0.13897678 0.1824032 ]
[0.9795551  0.94116971 0.13897678 0.1824032 ]
[0.9795551  0.9411697  0.13897679 0.1824032 ]
[0.9795551  0.9411697  0.13897678 0.18240321]
[1.         1.         0.75305529 0.79648171]
[0.99999999 1.         0.75305529 0.79648171]
[1.         0.99999999 0.75305529 0.79648171]
[1.         1.         0.7530553  0.79648171]
[1.         1.         0.75305529 0.79648172]
[0.98058675 0.97044268 0.5916779  0.63510432]
[0.98058676 0.97044268 0.5916779  0.63510432]
[0.98058675 0.97044269 0.5916779  0.63510432]
[0.98058675 0.97044268 0.59167791 0.63510432]
[0.98058675 0.97044268 0.5916779  0.63510433]
[0.99538463 0.99297294 0.71468887 0.75811529]
[0.99538464 0.99297294 0.71468887 0.75811529]
[0.99538463 0.99297295 0.71468887 0.75811529]
[0.99538463 0.99297294 0.71468888 0.75811529]
[0.99538463 0.99297294 0.71468887 0.7581153 ]
[0.9975804  0.99631607 0.73294176 0.77636818]
[0.99758041 0.99631607 0.73294176 

In [87]:
# perform the search
result = minimize(objective_ml, d, method='l-bfgs-b', jac=False, bounds=bfgs_bounds)
# summarize the result
print('Status : %s' % result['message'])
print('Total Evaluations: %d' % result['nfev'])
# evaluate solution
solution = result['x']
evaluation = objective_ml(solution)
print('Solution: f(%s) = %.5f' % (solution, evaluation))

[0.9795551  0.9411697  0.13897678 0.1824032 ]
[[0.9795551  0.9411697  0.13897678 0.1824032 ]]
[0.97955511 0.9411697  0.13897678 0.1824032 ]
[[0.97955511 0.9411697  0.13897678 0.1824032 ]]
[0.9795551  0.94116971 0.13897678 0.1824032 ]
[[0.9795551  0.94116971 0.13897678 0.1824032 ]]
[0.9795551  0.9411697  0.13897679 0.1824032 ]
[[0.9795551  0.9411697  0.13897679 0.1824032 ]]
[0.9795551  0.9411697  0.13897678 0.18240321]
[[0.9795551  0.9411697  0.13897678 0.18240321]]
Status : CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
Total Evaluations: 5
[0.9795551  0.9411697  0.13897678 0.1824032 ]
[[0.9795551  0.9411697  0.13897678 0.1824032 ]]
Solution: f([0.9795551  0.9411697  0.13897678 0.1824032 ]) = -0.60602
